## Pipeline: del ADN a la proteína 

En este ejercicio se implementa un pipeline biológico completo que reproduce, paso a paso, los tres procesos fundamentales del flujo de la información genética: replicación, transcripción y traducción.
El objetivo es entender cómo una secuencia de ADN puede transformarse, mediante reglas químicas precisas, en una proteína funcional.

### 1. Replicación del ADN

Este proceso simula lo que ocurre en la célula cuando la ADN polimerasa copia cada hebra original para formar una nueva molécula de ADN, como se ha hecho en los  ejercicios anteriores.
En el código, este paso se realiza utilizando la clase `Seq` de Biopython y el método `.complement()`, que genera automáticamente la cadena complementaria.
El script informa al usuario del progreso, mostrando tanto la hebra original como la hebra complementaria obtenida.

### 2. Transcripción (de ADN a ARNm)

Una vez obtenida la secuencia de ADN, se lleva a cabo la transcripción, en la que se genera una molécula de ARN mensajero (ARNm).
En términos computacionales, este paso se logra con el método `.transcribe()`, que sustituye las bases T (timina) por U (uracilo).

En el código se muestra claramente esta conversión, permitiendo observar cómo la secuencia resultante de ARNm mantiene la misma información que la hebra codificante, pero con uracilos en lugar de timinas.

Además, se invita a experimentar con el método .reverse_complement(), que genera la hebra reversa complementaria (la que simula la hebra molde), para comprobar cómo cambia la secuencia del ARNm dependiendo de la orientación de la hebra utilizada. Este análisis ayuda a comprender la importancia del sentido (5'→3') en los procesos genéticos.

### 3. Traducción (de ARNm a proteína)

Finalmente, se realiza la traducción de la secuencia de ARNm a una cadena de aminoácidos, empleando el método `.translate()`.
Este método interpreta los codones  según el código genético universal, generando una secuencia de letras que representan los aminoácidos correspondientes.
El proceso refleja lo que sucede en los ribosomas, donde el ARNm sirve de guía para la síntesis de proteínas.

El código muestra la proteína resultante, lo que permite comprobar cómo a partir de una secuencia inicial de ADN se puede obtener una proteína final, pasando por las etapas intermedias de replicación y transcripción.

In [1]:
from Bio import SeqIO
from Bio.Seq import Seq

def pipeline_adn_to_protein(adn_5a3):
    def titulo(txt):
        print("\n" + "="*70)
        print(txt)
        print("="*70)

    titulo("Codificante")
    print("Hebra coificante:", adn_5a3[:20], ("..." if len(adn_5a3) > 60 else ""))

    
    titulo("Replicación")
    complementaria_5a3 = adn_5a3.complement()
    rev_comp_5a3 = adn_5a3.reverse_complement()

    print("Complementaria (5'->3'):", complementaria_5a3[:20], ("..." if len(complementaria_5a3) > 20 else ""))
    print("Reversa complementaria (5'->3'):", rev_comp_5a3[:20], ("..." if len(rev_comp_5a3) > 20 else ""))

    hebra_molde_3a5 = rev_comp_5a3[::-1]
    print("Hebra molde (3'->5'):", hebra_molde_3a5[:20], ("..." if len(hebra_molde_3a5) > 20 else ""))

    titulo("Transcripción")
    mrna_5a3 = hebra_molde_3a5.complement().transcribe()

    print("ARNm (5'->3'):", mrna_5a3[:20], ("..." if len(mrna_5a3) > 20 else ""))

    titulo("Traducción")
    
    m = str(mrna_5a3)
    i = m.find("AUG")
    if i == -1:
        print("No hay AUG en el ARNm. No se puede iniciar traducción canónica.")
        prot_desde_AUG = None
    else:
        prot_desde_AUG = Seq(m[i:]).translate(to_stop=True)
        print("Posición del primer AUG:", i)

    prot_naive = mrna_5a3.translate()

    if prot_desde_AUG:
        if "*" in str(prot_naive):
            print("Aviso: se observan codones Stop en el recorrido naïve (esperable fuera de la ORF).")
    else:
        print("Sugerencia: revisa la orientación de la hebra de entrada o usa otra región con AUG.")

    return prot_desde_AUG, prot_naive

In [3]:
archivo = "data/gene.fna"

secuencias = []

for record in SeqIO.parse(archivo, "fasta"):
    secuencias.append(record)

print(f"Secuencias leídas: {len(secuencias)}")
print("Longitud de la primera secuencia:", len(secuencias[0].seq))

adn_5a3 = secuencias[0].seq
proteina_canónica, proteina_naive = pipeline_adn_to_protein(adn_5a3)

print("Proteína canónica (desde primer AUG):", proteina_canónica)
print("Proteína naïve (marco 0):", proteina_naive[:30])


Secuencias leídas: 2
Longitud de la primera secuencia: 19070

Codificante
Hebra coificante: CTCAAAAGTCTAGAGCCACC ...

Replicación
Complementaria (5'->3'): GAGTTTTCAGATCTCGGTGG ...
Reversa complementaria (5'->3'): TGGCAGCAAAGTTTTATTGT ...
Hebra molde (3'->5'): GAGTTTTCAGATCTCGGTGG ...

Transcripción
ARNm (5'->3'): CUCAAAAGUCUAGAGCCACC ...

Traducción
Posición del primer AUG: 135
Aviso: se observan codones Stop en el recorrido naïve (esperable fuera de la ORF).
Proteína canónica (desde primer AUG): MSPL
Proteína naïve (marco 0): LKSLEPPSREQVAAGLRGHFAFGLGACFPR


C:\Users\ralme\anaconda3\envs\BIO\Lib\site-packages\Bio\Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


##  Proteína canónica vs. Proteína naïve

La **proteína canónica** se obtiene mediante la traducción del **ARNm** desde el primer codón **AUG** (inicio de la traducción) hasta el primer codón **Stop** (final de la traducción). Este proceso sigue las reglas naturales de la **traducción** en la célula, donde solo se traduce la secuencia dentro del **marco de lectura abierto (ORF)**, es decir, la región que contiene el codón **AUG** y finaliza en el primer codón **Stop**.

Por otro lado, la **proteína naïve** se genera traduciendo todo el **ARNm** sin tener en cuenta los codones de inicio y paro. En este caso, el proceso no busca el primer **AUG**, sino que traduce la secuencia completa como si fuera un único bloque continuo, lo que puede resultar en una proteína truncada o con **codones de parada** internos, dependiendo de la secuencia.

* La **proteína canónica** es útil para obtener una versión biológicamente relevante de la proteína, ya que sigue las reglas naturales de la célula y respeta el **marco de lectura**.
* La **proteína naïve**, por su parte, se utiliza principalmente en contextos experimentales o de análisis, ya que permite obtener una traducción rápida y completa del **ARNm**. También se usa cuando se quiere estudiar el **marco completo** de lectura sin importar si hay codones **Stop** en el camino.

Ambas versiones son útiles dependiendo del objetivo: mientras que la **canónica** es la forma funcional de la proteína, la **naïve** ayuda a analizar el ARNm sin restricciones y puede ser útil para obtener una **visión general** del proceso de traducción.
